In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import (
    BertTokenizer, BertForSequenceClassification, BertModel,
    TrainingArguments, Trainer, DataCollatorWithPadding
)
from datasets import load_dataset, Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity
from scipy import stats
import re
import os
from tqdm import tqdm
import json
from datetime import datetime

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# =============================================================================
# LOAD BASELINE RESULTS FROM PART A
# =============================================================================

def load_baseline_results(results_dir="./bias_tracking_results"):
    """Load baseline bias propagation results from Part A"""
    
    print("📋 Loading baseline results from Part A...")
    
    trajectory_file = f"{results_dir}/bias_trajectory.json"
    if not os.path.exists(trajectory_file):
        print(f"❌ Baseline results not found at {results_dir}")
        print("Please run Part A first to establish baseline bias propagation.")
        return None
    
    with open(trajectory_file, 'r') as f:
        baseline_data = json.load(f)
    
    print(f"✅ Loaded baseline with {len(baseline_data['bias_trajectory'])} bias measurements")
    print(f"   Initial SEAT: {baseline_data['initial_bias_state']['seat_effect_size']:.4f}")
    print(f"   Final SEAT: {baseline_data['final_bias_state']['seat_effect_size']:.4f}")
    
    return baseline_data

# =============================================================================
# FAIRNESS REGULARIZATION IMPLEMENTATION
# =============================================================================

class FairnessBertForSequenceClassification(nn.Module):
    """
    BERT with fairness regularization
    
    Adds bias penalty terms to the loss function to encourage
    demographically fair predictions
    """
    
    def __init__(self, model_name='bert-base-uncased', num_labels=2, fairness_lambda=1.0):
        super(FairnessBertForSequenceClassification, self).__init__()
        
        # Load pre-trained BERT
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        
        # Fairness parameters
        self.fairness_lambda = fairness_lambda
        self.num_labels = num_labels
        
    def forward(self, input_ids, attention_mask=None, labels=None, gender_labels=None):
        """
        Forward pass with fairness regularization
        
        Args:
            input_ids: [batch_size, seq_len]
            attention_mask: [batch_size, seq_len]  
            labels: [batch_size] - sentiment labels
            gender_labels: [batch_size] - gender labels (0=male, 1=female, 2=neutral)
        """
        
        # Get BERT embeddings
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        
        # Sentiment classification
        logits = self.classifier(pooled_output)
        
        total_loss = 0
        task_loss = None
        fairness_loss = None
        
        if labels is not None:
            # Standard task loss
            loss_fct = nn.CrossEntropyLoss()
            task_loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            total_loss += task_loss
            
            # Fairness regularization
            if gender_labels is not None and self.fairness_lambda > 0:
                fairness_loss = self.calculate_fairness_penalty(
                    logits, labels, gender_labels
                )
                total_loss += self.fairness_lambda * fairness_loss
        
        return {
            'loss': total_loss,
            'logits': logits,
            'task_loss': task_loss,
            'fairness_loss': fairness_loss,
            'pooled_output': pooled_output
        }
    
    def calculate_fairness_penalty(self, logits, labels, gender_labels):
        """
        Calculate fairness penalty encouraging demographic parity
        
        Penalty = |P(y=1|male) - P(y=1|female)|^2
        """
        
        # Get predictions
        probs = torch.softmax(logits, dim=-1)
        positive_probs = probs[:, 1]  # Probability of positive sentiment
        
        # Separate by gender (0=male, 1=female, 2=neutral)
        male_mask = (gender_labels == 0)
        female_mask = (gender_labels == 1)
        
        # Calculate group-wise positive prediction rates
        if male_mask.sum() > 0 and female_mask.sum() > 0:
            male_positive_rate = positive_probs[male_mask].mean()
            female_positive_rate = positive_probs[female_mask].mean()
            
            # Demographic parity penalty
            parity_penalty = (male_positive_rate - female_positive_rate) ** 2
            
            return parity_penalty
        else:
            return torch.tensor(0.0, device=logits.device)

# =============================================================================
# DATA PREPARATION
# =============================================================================

def create_gender_labels(demographic_annotations):
    """Convert demographic strings to numeric labels"""
    gender_map = {'male': 0, 'female': 1, 'neutral': 2}
    return [gender_map[demo] for demo in demographic_annotations]

def prepare_fairness_dataset(annotated_dataset, tokenizer, max_length=128):
    """Prepare dataset with gender labels for fairness training"""
    
    def tokenize_and_add_gender(examples):
        # Tokenize
        tokenized = tokenizer(
            examples['sentence'],
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        
        # Add gender labels
        gender_labels = create_gender_labels(examples['demographic'])
        tokenized['gender_labels'] = gender_labels
        
        return tokenized
    
    tokenized_dataset = {}
    for split_name in ['train', 'validation']:
        if split_name in annotated_dataset:
            current_columns = annotated_dataset[split_name].column_names
            columns_to_remove = [col for col in current_columns 
                               if col not in ['label', 'demographic']]
            
            tokenized_dataset[split_name] = annotated_dataset[split_name].map(
                tokenize_and_add_gender,
                batched=True,
                remove_columns=columns_to_remove
            )
    
    return tokenized_dataset

class FairnessDataCollator:
    """Data collator for fairness training"""
    
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        
    def __call__(self, features):
        batch = {}
        batch['input_ids'] = torch.stack([torch.tensor(f['input_ids']) for f in features])
        batch['attention_mask'] = torch.stack([torch.tensor(f['attention_mask']) for f in features])
        batch['labels'] = torch.tensor([f['label'] for f in features])
        batch['gender_labels'] = torch.tensor([f['gender_labels'] for f in features])
        return batch

# =============================================================================
# FAIRNESS TRAINING WITH BIAS TRACKING
# =============================================================================

def run_fairness_experiment(fairness_lambda, annotated_dataset, seat_examples, 
                           demographic_test_sets, tokenizer, 
                           output_dir="./fairness_results"):
    """
    Run single fairness experiment with given lambda value
    """
    
    print(f"\n🔧 Running fairness experiment with λ={fairness_lambda}")
    
    # Prepare model
    model = FairnessBertForSequenceClassification(
        model_name='bert-base-uncased', 
        num_labels=2, 
        fairness_lambda=fairness_lambda
    )
    model.to(device)
    
    # Prepare data
    tokenized_dataset = prepare_fairness_dataset(annotated_dataset, tokenizer)
    train_dataset = tokenized_dataset['train']
    eval_dataset = tokenized_dataset['validation']
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"{output_dir}/lambda_{fairness_lambda}",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_steps=100,
        eval_steps=300,
        eval_strategy="steps",
        save_steps=300,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_accuracy",
        greater_is_better=True,
    )
    
    # Custom trainer with bias tracking
    class FairnessTrainer(Trainer):
        def __init__(self, *args, **kwargs):
            self.bias_measurements = []
            super().__init__(*args, **kwargs)
        
        def compute_loss(self, model, inputs, return_outputs=False):
            outputs = model(**inputs)
            return (outputs['loss'], outputs) if return_outputs else outputs['loss']
        
        def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
            # Standard evaluation
            eval_results = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
            
            # Add bias measurement
            bias_state = measure_current_bias_state(
                model.bert, tokenizer, seat_examples, demographic_test_sets
            )
            
            # Store bias measurement
            self.bias_measurements.append({
                'step': self.state.global_step,
                'epoch': self.state.epoch,
                'eval_accuracy': eval_results.get('eval_accuracy', 0),
                'seat_effect_size': bias_state['seat_effect_size'],
                'performance_gaps': bias_state['performance_gaps']
            })
            
            return eval_results
    
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return {"accuracy": accuracy_score(labels, predictions)}
    
    # Create trainer
    trainer = FairnessTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=FairnessDataCollator(tokenizer),
        compute_metrics=compute_metrics,
    )
    
    # Measure initial bias
    initial_bias = measure_current_bias_state(
        model.bert, tokenizer, seat_examples, demographic_test_sets
    )
    
    # Train model
    print(f"   Training with fairness regularization...")
    trainer.train()
    
    # Final evaluation
    final_eval = trainer.evaluate()
    final_bias = measure_current_bias_state(
        model.bert, tokenizer, seat_examples, demographic_test_sets
    )
    
    # Comprehensive demographic evaluation
    final_demographic_performance = evaluate_comprehensive_performance(
        model, tokenizer, demographic_test_sets
    )
    
    experiment_results = {
        'fairness_lambda': fairness_lambda,
        'initial_bias': initial_bias,
        'final_bias': final_bias,
        'final_accuracy': final_eval['eval_accuracy'],
        'bias_measurements': trainer.bias_measurements,
        'demographic_performance': final_demographic_performance,
        'bias_reduction': initial_bias['seat_effect_size'] - final_bias['seat_effect_size']
    }
    
    print(f"   Final Accuracy: {final_eval['eval_accuracy']:.4f}")
    print(f"   Final SEAT Bias: {final_bias['seat_effect_size']:.4f}")
    print(f"   Bias Reduction: {experiment_results['bias_reduction']:.4f}")
    
    return experiment_results

# =============================================================================
# BIAS MEASUREMENT FUNCTIONS (from Part A)
# =============================================================================

def get_sentence_embedding(text, model, tokenizer, pooling='cls'):
    """Extract sentence embedding from BERT model"""
    inputs = tokenizer(text, return_tensors='pt', padding=True, 
                      truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        hidden_states = outputs.last_hidden_state[0]
    
    if pooling == 'cls':
        return hidden_states[0].cpu().numpy()
    elif pooling == 'mean':
        attention_mask = inputs['attention_mask'][0]
        masked_embeddings = hidden_states * attention_mask.unsqueeze(-1)
        return (masked_embeddings.sum(dim=0) / attention_mask.sum()).cpu().numpy()

def calculate_mini_seat_score(seat_examples, model, tokenizer):
    """Calculate SEAT bias score"""
    
    # Get embeddings for all groups
    male_embeddings = [get_sentence_embedding(sent, model, tokenizer) 
                      for sent in seat_examples['male_targets']]
    female_embeddings = [get_sentence_embedding(sent, model, tokenizer) 
                        for sent in seat_examples['female_targets']]
    career_embeddings = [get_sentence_embedding(sent, model, tokenizer) 
                        for sent in seat_examples['career_attributes']]
    family_embeddings = [get_sentence_embedding(sent, model, tokenizer) 
                        for sent in seat_examples['family_attributes']]
    
    # Calculate association scores
    male_scores = []
    female_scores = []
    
    for male_emb in male_embeddings:
        career_sim = np.mean([cosine_similarity([male_emb], [career_emb])[0][0] 
                             for career_emb in career_embeddings])
        family_sim = np.mean([cosine_similarity([male_emb], [family_emb])[0][0] 
                             for family_emb in family_embeddings])
        male_scores.append(career_sim - family_sim)
    
    for female_emb in female_embeddings:
        career_sim = np.mean([cosine_similarity([female_emb], [career_emb])[0][0] 
                             for career_emb in career_embeddings])
        family_sim = np.mean([cosine_similarity([female_emb], [family_emb])[0][0] 
                             for family_emb in family_embeddings])
        female_scores.append(career_sim - family_sim)
    
    # Calculate effect size
    mean_diff = np.mean(male_scores) - np.mean(female_scores)
    pooled_std = np.sqrt((np.var(male_scores) + np.var(female_scores)) / 2)
    effect_size = mean_diff / pooled_std if pooled_std > 0 else 0
    
    return effect_size, male_scores, female_scores

def evaluate_comprehensive_performance(model, tokenizer, demographic_test_sets):
    """Comprehensive evaluation across demographic groups"""
    
    results = {}
    
    for demographic, test_data in demographic_test_sets.items():
        if len(test_data) == 0:
            continue
            
        predictions = []
        true_labels = []
        confidences = []
        
        model.eval()
        with torch.no_grad():
            for example in test_data:
                inputs = tokenizer(
                    example['sentence'],
                    return_tensors='pt',
                    padding=True,
                    truncation=True,
                    max_length=512
                )
                inputs = {k: v.to(model.device) for k, v in inputs.items()}
                
                # Handle both fairness model and standard BERT
                if hasattr(model, 'bert'):
                    # Fairness model
                    outputs = model(**inputs)
                    logits = outputs['logits']
                else:
                    # Standard BERT
                    outputs = model(**inputs)
                    logits = outputs.logits
                
                probs = torch.softmax(logits, dim=-1)
                prediction = torch.argmax(logits, dim=-1).item()
                confidence = torch.max(probs).item()
                
                predictions.append(prediction)
                true_labels.append(example['label'])
                confidences.append(confidence)
        
        accuracy = accuracy_score(true_labels, predictions)
        avg_confidence = np.mean(confidences)
        
        # Calculate positive prediction rate
        positive_rate = np.mean([1 for pred in predictions if pred == 1])
        
        results[demographic] = {
            'accuracy': accuracy,
            'avg_confidence': avg_confidence,
            'positive_rate': positive_rate,
            'n_examples': len(test_data)
        }
    
    return results

def measure_current_bias_state(model, tokenizer, seat_examples, demographic_test_sets):
    """Measure current bias state"""
    
    # SEAT measurement
    seat_effect_size, male_scores, female_scores = calculate_mini_seat_score(
        seat_examples, model, tokenizer
    )
    
    # Demographic performance
    demographic_results = evaluate_comprehensive_performance(
        model, tokenizer, demographic_test_sets
    )
    
    # Performance gaps
    performance_gaps = {}
    if 'male' in demographic_results and 'female' in demographic_results:
        male_acc = demographic_results['male']['accuracy']
        female_acc = demographic_results['female']['accuracy']
        male_conf = demographic_results['male']['avg_confidence']
        female_conf = demographic_results['female']['avg_confidence']
        male_pos_rate = demographic_results['male']['positive_rate']
        female_pos_rate = demographic_results['female']['positive_rate']
        
        performance_gaps = {
            'accuracy_gap': male_acc - female_acc,
            'confidence_gap': male_conf - female_conf,
            'positive_rate_gap': male_pos_rate - female_pos_rate,
            'male_accuracy': male_acc,
            'female_accuracy': female_acc
        }
    
    return {
        'seat_effect_size': seat_effect_size,
        'seat_male_scores': male_scores,
        'seat_female_scores': female_scores,
        'demographic_performance': demographic_results,
        'performance_gaps': performance_gaps
    }

# =============================================================================
# ACCURACY VS BIAS TRADE-OFF ANALYSIS
# =============================================================================

def run_trade_off_analysis(baseline_data, output_dir="./fairness_analysis"):
    """
    Run comprehensive accuracy vs bias trade-off analysis
    """
    
    print("🚀 STARTING ACCURACY VS BIAS TRADE-OFF ANALYSIS")
    print("="*60)
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Setup from baseline (reuse Part A setup)
    print("📋 Setting up data and models...")
    
    # Import data setup functions from Part A
    from datasets import load_dataset
    
    # Recreate the same setup as Part A
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    # Load and prepare data (same as Part A)
    sst2_dataset = load_dataset("glue", "sst2")
    male_names, female_names = create_demographic_names()
    annotated_dataset = create_demographic_annotated_dataset(sst2_dataset, male_names, female_names)
    demographic_test_sets, annotated_dataset = create_demographic_test_sets(annotated_dataset)
    seat_examples = create_mini_seat_examples()
    
    # Define fairness lambda values to test
    lambda_values = [0.0, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
    
    print(f"🔬 Testing {len(lambda_values)} fairness regularization strengths...")
    print(f"   Lambda values: {lambda_values}")
    
    # Run experiments
    all_results = []
    
    for i, lambda_val in enumerate(lambda_values):
        print(f"\n📊 Experiment {i+1}/{len(lambda_values)}: λ = {lambda_val}")
        
        experiment_result = run_fairness_experiment(
            fairness_lambda=lambda_val,
            annotated_dataset=annotated_dataset,
            seat_examples=seat_examples,
            demographic_test_sets=demographic_test_sets,
            tokenizer=tokenizer,
            output_dir=output_dir
        )
        
        all_results.append(experiment_result)
    
    # Add baseline to results
    baseline_result = {
        'fairness_lambda': 'baseline',
        'final_accuracy': baseline_data['final_bias_state']['demographic_performance']['all']['accuracy'] if 'all' in baseline_data['final_bias_state']['demographic_performance'] else 0.85,  # fallback
        'final_bias': {'seat_effect_size': baseline_data['final_bias_state']['seat_effect_size']},
        'bias_reduction': 0  # baseline has no bias reduction
    }
    all_results.append(baseline_result)
    
    # Analyze and visualize trade-offs
    print("\n📈 Analyzing trade-offs...")
    trade_off_analysis = analyze_accuracy_bias_trade_offs(all_results, baseline_data, output_dir)
    
    # Save complete results
    with open(f"{output_dir}/trade_off_results.json", 'w') as f:
        json.dump({
            'lambda_values': lambda_values,
            'all_results': all_results,
            'trade_off_analysis': trade_off_analysis,
            'baseline_data': baseline_data
        }, f, indent=2, default=str)
    
    print(f"\n✅ Trade-off analysis complete!")
    print(f"📁 Results saved to: {output_dir}/")
    
    return all_results, trade_off_analysis

def analyze_accuracy_bias_trade_offs(all_results, baseline_data, output_dir):
    """Analyze and visualize accuracy vs bias trade-offs"""
    
    # Extract data for analysis
    lambda_vals = []
    accuracies = []
    bias_scores = []
    bias_reductions = []
    
    for result in all_results:
        if result['fairness_lambda'] != 'baseline':
            lambda_vals.append(result['fairness_lambda'])
            accuracies.append(result['final_accuracy'])
            bias_scores.append(result['final_bias']['seat_effect_size'])
            bias_reductions.append(result['bias_reduction'])
    
    # Baseline values
    baseline_accuracy = all_results[-1]['final_accuracy']
    baseline_bias = all_results[-1]['final_bias']['seat_effect_size']
    
    # Create comprehensive visualization
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: Accuracy vs Lambda
    ax1.plot(lambda_vals, accuracies, 'bo-', linewidth=2, markersize=8, label='Fairness Regularization')
    ax1.axhline(y=baseline_accuracy, color='red', linestyle='--', linewidth=2, label='Baseline (No Regularization)')
    ax1.set_xlabel('Fairness Regularization Strength (λ)')
    ax1.set_ylabel('Accuracy')
    ax1.set_title('Accuracy vs Fairness Regularization')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    ax1.set_xscale('log')
    
    # Plot 2: Bias vs Lambda
    ax2.plot(lambda_vals, bias_scores, 'ro-', linewidth=2, markersize=8, label='Fairness Regularization')
    ax2.axhline(y=baseline_bias, color='red', linestyle='--', linewidth=2, label='Baseline (No Regularization)')
    ax2.set_xlabel('Fairness Regularization Strength (λ)')
    ax2.set_ylabel('SEAT Effect Size (Bias)')
    ax2.set_title('Bias vs Fairness Regularization')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    ax2.set_xscale('log')
    
    # Plot 3: Trade-off Curve (Accuracy vs Bias)
    ax3.scatter(bias_scores, accuracies, c=lambda_vals, s=100, cmap='viridis', alpha=0.8)
    ax3.scatter(baseline_bias, baseline_accuracy, c='red', s=150, marker='*', label='Baseline')
    
    # Add lambda labels
    for i, (bias, acc, lam) in enumerate(zip(bias_scores, accuracies, lambda_vals)):
        ax3.annotate(f'λ={lam}', (bias, acc), xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    ax3.set_xlabel('SEAT Effect Size (Bias)')
    ax3.set_ylabel('Accuracy')
    ax3.set_title('Accuracy vs Bias Trade-off Curve')
    ax3.grid(True, alpha=0.3)
    ax3.legend()
    
    # Add colorbar
    cbar = plt.colorbar(ax3.collections[0], ax=ax3)
    cbar.set_label('λ (Fairness Strength)')
    
    # Plot 4: Bias Reduction vs Accuracy Loss
    accuracy_losses = [baseline_accuracy - acc for acc in accuracies]
    
    ax4.scatter(bias_reductions, accuracy_losses, c=lambda_vals, s=100, cmap='plasma', alpha=0.8)
    ax4.set_xlabel('Bias Reduction (SEAT Effect Size)')
    ax4.set_ylabel('Accuracy Loss')
    ax4.set_title('Bias Reduction vs Accuracy Loss')
    ax4.grid(True, alpha=0.3)
    
    # Add efficiency frontier
    if len(bias_reductions) > 1:
        # Find Pareto frontier (best trade-offs)
        frontier_indices = []
        for i, (bias_red, acc_loss) in enumerate(zip(bias_reductions, accuracy_losses)):
            is_dominated = False
            for j, (other_bias_red, other_acc_loss) in enumerate(zip(bias_reductions, accuracy_losses)):
                if i != j and other_bias_red >= bias_red and other_acc_loss <= acc_loss and (other_bias_red > bias_red or other_acc_loss < acc_loss):
                    is_dominated = True
                    break
            if not is_dominated:
                frontier_indices.append(i)
        
        if len(frontier_indices) > 1:
            frontier_bias = [bias_reductions[i] for i in frontier_indices]
            frontier_acc_loss = [accuracy_losses[i] for i in frontier_indices]
            # Sort by bias reduction
            sorted_pairs = sorted(zip(frontier_bias, frontier_acc_loss))
            frontier_bias, frontier_acc_loss = zip(*sorted_pairs)
            ax4.plot(frontier_bias, frontier_acc_loss, 'r--', alpha=0.7, label='Pareto Frontier')
            ax4.legend()
    
    # Add lambda labels
    for i, (bias_red, acc_loss, lam) in enumerate(zip(bias_reductions, accuracy_losses, lambda_vals)):
        ax4.annotate(f'λ={lam}', (bias_red, acc_loss), xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/accuracy_bias_trade_off.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    # Statistical analysis
    print("\n📊 TRADE-OFF ANALYSIS RESULTS")
    print("="*50)
    
    # Find best trade-offs
    best_bias_reduction_idx = np.argmax(bias_reductions)
    best_accuracy_idx = np.argmax(accuracies)
    
    # Calculate efficiency metric (bias reduction per accuracy loss)
    efficiency_scores = []
    for bias_red, acc_loss in zip(bias_reductions, accuracy_losses):
        if acc_loss > 0:
            efficiency_scores.append(bias_red / acc_loss)
        else:
            efficiency_scores.append(float('inf') if bias_red > 0 else 0)
    
    best_efficiency_idx = np.argmax(efficiency_scores)
    
    print(f"🎯 OPTIMAL CONFIGURATIONS:")
    print(f"   Best Bias Reduction: λ={lambda_vals[best_bias_reduction_idx]:.1f}")
    print(f"      Bias Reduction: {bias_reductions[best_bias_reduction_idx]:.4f}")
    print(f"      Accuracy: {accuracies[best_bias_reduction_idx]:.4f}")
    print(f"      Accuracy Loss: {accuracy_losses[best_bias_reduction_idx]:.4f}")
    
    print(f"\n   Best Accuracy: λ={lambda_vals[best_accuracy_idx]:.1f}")
    print(f"      Accuracy: {accuracies[best_accuracy_idx]:.4f}")
    print(f"      Bias Reduction: {bias_reductions[best_accuracy_idx]:.4f}")
    
    print(f"\n   Best Efficiency: λ={lambda_vals[best_efficiency_idx]:.1f}")
    print(f"      Efficiency Score: {efficiency_scores[best_efficiency_idx]:.2f}")
    print(f"      Bias Reduction: {bias_reductions[best_efficiency_idx]:.4f}")
    print(f"      Accuracy Loss: {accuracy_losses[best_efficiency_idx]:.4f}")
    
    # Summary statistics
    max_bias_reduction = max(bias_reductions)
    min_accuracy_loss = min(accuracy_losses)
    
    print(f"\n📈 SUMMARY STATISTICS:")
    print(f"   Baseline Bias: {baseline_bias:.4f}")
    print(f"   Baseline Accuracy: {baseline_accuracy:.4f}")
    print(f"   Max Bias Reduction: {max_bias_reduction:.4f}")
    print(f"   Min Accuracy Loss: {min_accuracy_loss:.4f}")
    print(f"   Bias Reduction Range: {min(bias_reductions):.4f} to {max_bias_reduction:.4f}")
    print(f"   Accuracy Range: {min(accuracies):.4f} to {max(accuracies):.4f}")
    
    # Calculate correlation between lambda and outcomes
    lambda_bias_corr = np.corrcoef(lambda_vals, bias_reductions)[0, 1]
    lambda_acc_corr = np.corrcoef(lambda_vals, accuracies)[0, 1]
    
    print(f"\n🔗 CORRELATIONS:")
    print(f"   λ vs Bias Reduction: {lambda_bias_corr:.3f}")
    print(f"   λ vs Accuracy: {lambda_acc_corr:.3f}")
    
    # Recommendations
    print(f"\n💡 RECOMMENDATIONS:")
    
    if max_bias_reduction > 0.5:
        print(f"   ✅ Fairness regularization is EFFECTIVE - achieves substantial bias reduction")
    else:
        print(f"   ⚠️ Fairness regularization has LIMITED effectiveness")
    
    if min_accuracy_loss < 0.02:
        print(f"   ✅ Bias reduction can be achieved with MINIMAL accuracy loss")
    else:
        print(f"   ⚠️ Bias reduction comes with SIGNIFICANT accuracy trade-offs")
    
    # Find sweet spot
    sweet_spot_idx = best_efficiency_idx
    sweet_spot_lambda = lambda_vals[sweet_spot_idx]
    
    print(f"   🎯 RECOMMENDED λ = {sweet_spot_lambda:.1f} for best bias-accuracy balance")
    
    trade_off_analysis = {
        'lambda_values': lambda_vals,
        'accuracies': accuracies,
        'bias_scores': bias_scores,
        'bias_reductions': bias_reductions,
        'accuracy_losses': accuracy_losses,
        'efficiency_scores': efficiency_scores,
        'best_configs': {
            'best_bias_reduction': {
                'lambda': lambda_vals[best_bias_reduction_idx],
                'bias_reduction': bias_reductions[best_bias_reduction_idx],
                'accuracy': accuracies[best_bias_reduction_idx]
            },
            'best_accuracy': {
                'lambda': lambda_vals[best_accuracy_idx],
                'accuracy': accuracies[best_accuracy_idx],
                'bias_reduction': bias_reductions[best_accuracy_idx]
            },
            'best_efficiency': {
                'lambda': lambda_vals[best_efficiency_idx],
                'efficiency_score': efficiency_scores[best_efficiency_idx],
                'bias_reduction': bias_reductions[best_efficiency_idx],
                'accuracy_loss': accuracy_losses[best_efficiency_idx]
            }
        },
        'correlations': {
            'lambda_bias_correlation': lambda_bias_corr,
            'lambda_accuracy_correlation': lambda_acc_corr
        },
        'summary_stats': {
            'max_bias_reduction': max_bias_reduction,
            'min_accuracy_loss': min_accuracy_loss,
            'baseline_bias': baseline_bias,
            'baseline_accuracy': baseline_accuracy
        }
    }
    
    return trade_off_analysis

# =============================================================================
# HELPER FUNCTIONS (imported from Part A logic)
# =============================================================================

def create_demographic_names():
    """Create lists of male and female names for demographic analysis"""
    male_names = [
        "James", "John", "Robert", "Michael", "William", "David", "Richard", 
        "Joseph", "Thomas", "Christopher", "Charles", "Daniel", "Matthew", 
        "Anthony", "Mark", "Donald", "Steven", "Paul", "Andrew", "Joshua"
    ]
    
    female_names = [
        "Mary", "Patricia", "Jennifer", "Linda", "Elizabeth", "Barbara", 
        "Susan", "Jessica", "Sarah", "Karen", "Nancy", "Lisa", "Betty", 
        "Helen", "Sandra", "Donna", "Carol", "Ruth", "Sharon", "Michelle"
    ]
    
    return male_names, female_names

def detect_demographic_mentions(text, male_names, female_names):
    """Detect if text contains demographic mentions (names or pronouns)"""
    text_lower = text.lower()
    
    # Check for names
    for name in male_names:
        if name.lower() in text_lower:
            return 'male'
    
    for name in female_names:
        if name.lower() in text_lower:
            return 'female'
    
    # Check for pronouns
    male_pronouns = ['he', 'his', 'him', 'himself']
    female_pronouns = ['she', 'her', 'hers', 'herself']
    
    # Use word boundaries to avoid partial matches
    for pronoun in male_pronouns:
        if re.search(r'\b' + pronoun + r'\b', text_lower):
            return 'male'
    
    for pronoun in female_pronouns:
        if re.search(r'\b' + pronoun + r'\b', text_lower):
            return 'female'
    
    return 'neutral'

def create_demographic_annotated_dataset(dataset, male_names, female_names):
    """Add demographic annotations to SST-2 dataset"""
    
    def add_demographic_info(example):
        demographic = detect_demographic_mentions(example['sentence'], male_names, female_names)
        example['demographic'] = demographic
        return example
    
    # Annotate all splits
    annotated_dataset = {}
    for split_name in dataset.keys():
        annotated_dataset[split_name] = dataset[split_name].map(add_demographic_info)
    
    return annotated_dataset

def create_demographic_test_sets(annotated_dataset, test_split_ratio=0.15):
    """Create separate test sets for demographic analysis"""
    from sklearn.model_selection import train_test_split
    
    # Convert training data for splitting
    train_data = annotated_dataset['train']
    
    # Extract data for splitting
    sentences = list(train_data['sentence'])
    labels = list(train_data['label'])
    demographics = list(train_data['demographic'])
    
    # Stratified split to maintain label distribution
    train_sentences, test_sentences, train_labels, test_labels, train_demos, test_demos = train_test_split(
        sentences, labels, demographics,
        test_size=test_split_ratio,
        stratify=labels,
        random_state=42
    )
    
    # Create new train/test datasets
    new_train_data = Dataset.from_dict({
        'sentence': train_sentences,
        'label': train_labels,
        'demographic': train_demos
    })
    
    test_data = Dataset.from_dict({
        'sentence': test_sentences,
        'label': test_labels,
        'demographic': test_demos
    })
    
    # Split test data by demographic groups
    male_examples = test_data.filter(lambda x: x['demographic'] == 'male')
    female_examples = test_data.filter(lambda x: x['demographic'] == 'female')
    neutral_examples = test_data.filter(lambda x: x['demographic'] == 'neutral')
    
    demographic_splits = {
        'male': male_examples,
        'female': female_examples,
        'neutral': neutral_examples,
        'all': test_data
    }
    
    # Update the annotated dataset with new split
    annotated_dataset['train'] = new_train_data
    annotated_dataset['test'] = test_data
    
    return demographic_splits, annotated_dataset

def create_mini_seat_examples():
    """Create lightweight SEAT examples for frequent bias measurement"""
    
    # Target groups: Male and female names in simple contexts
    male_names = ["James", "John", "Robert", "Michael"]
    female_names = ["Mary", "Patricia", "Jennifer", "Linda"]
    
    male_sentences = [f"{name} is a person." for name in male_names]
    female_sentences = [f"{name} is a person." for name in female_names]
    
    # Attribute groups: Career vs. family oriented (simplified)
    career_attributes = [
        "The executive makes decisions.",
        "The engineer solves problems.", 
        "The manager leads teams.",
        "The analyst studies data."
    ]
    
    family_attributes = [
        "The caregiver helps others.",
        "The teacher nurtures students.",
        "The nurse provides care.",
        "The assistant offers support."
    ]
    
    return {
        'male_targets': male_sentences,
        'female_targets': female_sentences,
        'career_attributes': career_attributes,
        'family_attributes': family_attributes
    }

# =============================================================================
# MAIN EXECUTION FUNCTION
# =============================================================================

def run_complete_fairness_analysis(baseline_dir="./bias_tracking_results", 
                                  output_dir="./fairness_analysis"):
    """
    Run complete fairness regularization analysis
    
    This function:
    1. Loads baseline results from Part A
    2. Tests multiple fairness regularization strengths
    3. Analyzes accuracy vs bias trade-offs
    4. Provides recommendations
    """
    
    print("🚀 STARTING COMPLETE FAIRNESS ANALYSIS")
    print("="*60)
    
    # Load baseline results
    baseline_data = load_baseline_results(baseline_dir)
    if baseline_data is None:
        return None
    
    # Run trade-off analysis
    all_results, trade_off_analysis = run_trade_off_analysis(baseline_data, output_dir)
    
    print("\n🎉 FAIRNESS ANALYSIS COMPLETE!")
    print("="*40)
    print(f"📁 Results saved to: {output_dir}/")
    print(f"📊 Trade-off plots: {output_dir}/accuracy_bias_trade_off.png")
    print(f"📋 Full results: {output_dir}/trade_off_results.json")
    
    # Quick summary
    best_config = trade_off_analysis['best_configs']['best_efficiency']
    print(f"\n🎯 QUICK SUMMARY:")
    print(f"   Recommended λ: {best_config['lambda']}")
    print(f"   Bias Reduction: {best_config['bias_reduction']:.4f}")
    print(f"   Accuracy Loss: {best_config['accuracy_loss']:.4f}")
    
    return {
        'all_results': all_results,
        'trade_off_analysis': trade_off_analysis,
        'baseline_data': baseline_data
    }

# =============================================================================
# EXAMPLE USAGE
# =============================================================================

if __name__ == "__main__":
    # Run complete fairness analysis
    print("Starting fairness regularization analysis...")
    
    results = run_complete_fairness_analysis(
        baseline_dir="./bias_tracking_results",
        output_dir="./fairness_analysis"
    )
    
    if results:
        print("\n✅ Analysis complete!")
        print("Check the output directory for detailed results and visualizations.")
    else:
        print("\n❌ Analysis failed. Please run Part A first.")